In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 8

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda() 
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 1000
    xa1 = 2*torch.rand(Nb, dimension) - 1.0
    xa1[:, 0] = -torch.ones(Nb)
    
    xa2 = 2*torch.rand(Nb, dimension) - 1.0
    xa2[:, 0:8] = xa1[:, 0:8]
    xa2[:, 0] = torch.ones(Nb)
    
    xb1 = 2*torch.rand(Nb, dimension) - 1.0
    xb1[:, 1] = -torch.ones(Nb)
    
    xb2 = 2*torch.rand(Nb, dimension) - 1.0
    xb2[:, 0:8]  = xb1[:, 0:8] 
    xb2[:, 1] = torch.ones(Nb)
    
    xc1 = 2*torch.rand(Nb, dimension) - 1.0
    xc1[:, 2] = -torch.ones(Nb)
    
    xc2 = 2*torch.rand(Nb, dimension) - 1.0
    xc2[:, 0:8] = xc1[:, 0:8]
    xc2[:, 2] = torch.ones(Nb)
    
    xd1 = 2*torch.rand(Nb, dimension) - 1.0
    xd1[:, 3] = -torch.ones(Nb)
    
    xd2 = 2*torch.rand(Nb, dimension) - 1.0
    xd2[:, 0:8] = xd1[:, 0:8]
    xd2[:, 3] = torch.ones(Nb)
    
    xe1 = 2*torch.rand(Nb, dimension) - 1.0
    xe1[:, 4] = -torch.ones(Nb)
    
    xe2 = 2*torch.rand(Nb, dimension) - 1.0
    xe2[:, 0:8] = xe1[:, 0:8]
    xe2[:, 4] = torch.ones(Nb)
    
    xf1 = 2*torch.rand(Nb, dimension) - 1.0
    xf1[:, 5] = -torch.ones(Nb)
    
    xf2 = 2*torch.rand(Nb, dimension) - 1.0
    xf2[:, 0:8] = xf1[:, 0:8] 
    xf2[:, 5] = torch.ones(Nb)
    
    xg1 = 2*torch.rand(Nb, dimension) - 1.0
    xg1[:, 6] = -torch.ones(Nb)
    
    xg2 = 2*torch.rand(Nb, dimension) - 1.0
    xg2[:, 0:8] = xg1[:, 0:8] 
    xg2[:, 6] = torch.ones(Nb)
    
    xh1 = 2*torch.rand(Nb, dimension) - 1.0
    xh1[:, 7] = -torch.ones(Nb)
    
    xh2 = 2*torch.rand(Nb, dimension) - 1.0
    xh2[:, 0:8] = xh1[:, 0:8]
    xh2[:, 7] = torch.ones(Nb)
    
    xa1 = xa1.cuda()
    xa2 = xa2.cuda()
    xb1 = xb1.cuda()
    xb2 = xb2.cuda()
    xc1 = xc1.cuda()
    xc2 = xc2.cuda()
    xd1 = xd1.cuda()
    xd2 = xd2.cuda()
    xe1 = xe1.cuda()
    xe2 = xe2.cuda()
    xf1 = xf1.cuda()
    xf2 = xf2.cuda()
    xg1 = xg1.cuda()
    xg2 = xg2.cuda()
    xh1 = xh1.cuda()
    xh2 = xh2.cuda()
    
    pxa = ((model(xa1) - model(xa2))**2).cuda() 
    pxb = ((model(xb1) - model(xb2))**2).cuda()  
    pxc = ((model(xc1) - model(xc2))**2).cuda()  
    pxd = ((model(xd1) - model(xd2))**2).cuda()  
    pxe = ((model(xe1) - model(xe2))**2).cuda() 
    pxf = ((model(xf1) - model(xf2))**2).cuda()  
    pxg = ((model(xg1) - model(xg2))**2).cuda()  
    pxh = ((model(xh1) - model(xh2))**2).cuda()
    
    part_2 = torch.sum((pxa + pxb + pxc + pxd + pxe + pxf + pxg + pxh)[:,0])/2/xa1.size()[0]/8
    
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda()  
    dx4[:, 3] = torch.ones(xb1.size()[0])
    dx5 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx5[:, 4] = torch.ones(xb1.size()[0])
    dx6 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx6[:, 5] = torch.ones(xb1.size()[0])
    dx7 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx7[:, 6] = torch.ones(xb1.size()[0])
    dx8 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx8[:, 7] = torch.ones(xb1.size()[0])
    
    dxa = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 - (model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2)**2).cuda()  
    dxb = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 - (model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2)**2).cuda()  
    dxc = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 - (model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2)**2).cuda()  
    dxd = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 - (model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2)**2).cuda()  
    dxe = (((model(xe1+step_size*dx5) - model(xe1-step_size*dx5))/step_size/2 - (model(xe2+step_size*dx5) - model(xe2-step_size*dx5))/step_size/2)**2).cuda()  
    dxf = (((model(xf1+step_size*dx6) - model(xf1-step_size*dx6))/step_size/2 - (model(xf2+step_size*dx6) - model(xf2-step_size*dx6))/step_size/2)**2).cuda()  
    dxg = (((model(xg1+step_size*dx7) - model(xg1-step_size*dx7))/step_size/2 - (model(xg2+step_size*dx7) - model(xg2-step_size*dx7))/step_size/2)**2).cuda()  
    dxh = (((model(xh1+step_size*dx8) - model(xh1-step_size*dx8))/step_size/2 - (model(xh2+step_size*dx8) - model(xh2-step_size*dx8))/step_size/2)**2).cuda()  
    
    
    part_3 = torch.sum((dxa + dxb + dxc + dxd + dxe + dxf + dxg + dxh)[:,0])/2/xb1.size()[0]/8
    
    lambda1 = 1.0
    lambda2 = 0.5
    return part_1 + lambda1 * part_2 + lambda2 * part_3

In [10]:
Data_size = 4000
def Gendata():
    x = 2*torch.rand(Data_size, dimension) - 1.0
    return x.cuda()  

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 16)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=8, out_features=16, bias=True)
  (linear1): Linear(in_features=16, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=16, bias=True)
  (linear3): Linear(in_features=16, out_features=16, bias=True)
  (linear4): Linear(in_features=16, out_features=16, bias=True)
  (linear5): Linear(in_features=16, out_features=16, bias=True)
  (linear6): Linear(in_features=16, out_features=16, bias=True)
  (linear_out): Linear(in_features=16, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [16]:
traintime = 50000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_8D_Periodic.npy", error_save)
np.save("DGM_relative_error_8D_Periodic.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(38139.1853, device='cuda:0')
current relative error is:  tensor(6.4040, device='cuda:0')
current epoch is:  50
current loss is:  tensor(6464.7852, device='cuda:0')
current relative error is:  tensor(2.2620, device='cuda:0')
current epoch is:  100
current loss is:  tensor(3538.1715, device='cuda:0')
current relative error is:  tensor(1.7105, device='cuda:0')
current epoch is:  150
current loss is:  tensor(2571.3558, device='cuda:0')
current relative error is:  tensor(1.4617, device='cuda:0')
current epoch is:  200
current loss is:  tensor(2126.5019, device='cuda:0')
current relative error is:  tensor(1.3394, device='cuda:0')
current epoch is:  250
current loss is:  tensor(1819.4023, device='cuda:0')
current relative error is:  tensor(1.2236, device='cuda:0')
current epoch is:  300
current loss is:  tensor(1627.9644, device='cuda:0')
current relative error is:  tensor(1.2108, device='cuda:0')
current epoch is:  350
current loss is:  tensor(14

current epoch is:  3050
current loss is:  tensor(186.7622, device='cuda:0')
current relative error is:  tensor(2.2517, device='cuda:0')
current epoch is:  3100
current loss is:  tensor(188.0120, device='cuda:0')
current relative error is:  tensor(2.3775, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(182.4848, device='cuda:0')
current relative error is:  tensor(2.3114, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(177.8988, device='cuda:0')
current relative error is:  tensor(2.3407, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(178.2339, device='cuda:0')
current relative error is:  tensor(2.3320, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(175.4396, device='cuda:0')
current relative error is:  tensor(2.3963, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(177.5495, device='cuda:0')
current relative error is:  tensor(2.3501, device='cuda:0')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(62.5065, device='cuda:0')
current relative error is:  tensor(1.7394, device='cuda:0')
current epoch is:  6150
current loss is:  tensor(59.2813, device='cuda:0')
current relative error is:  tensor(1.7729, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(56.5608, device='cuda:0')
current relative error is:  tensor(1.7516, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(56.0388, device='cuda:0')
current relative error is:  tensor(1.6921, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(54.1947, device='cuda:0')
current relative error is:  tensor(1.6439, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(51.6907, device='cuda:0')
current relative error is:  tensor(1.6929, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(49.7090, device='cuda:0')
current relative error is:  tensor(1.6152, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(47.818

current epoch is:  9150
current loss is:  tensor(15.2577, device='cuda:0')
current relative error is:  tensor(1.0898, device='cuda:0')
current epoch is:  9200
current loss is:  tensor(14.8108, device='cuda:0')
current relative error is:  tensor(1.0996, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(14.5743, device='cuda:0')
current relative error is:  tensor(1.0561, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(14.2947, device='cuda:0')
current relative error is:  tensor(1.0674, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(14.5245, device='cuda:0')
current relative error is:  tensor(1.0189, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(13.8737, device='cuda:0')
current relative error is:  tensor(1.0438, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(14.1584, device='cuda:0')
current relative error is:  tensor(1.0705, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(13.720

current epoch is:  12200
current loss is:  tensor(7.5652, device='cuda:0')
current relative error is:  tensor(0.6626, device='cuda:0')
current epoch is:  12250
current loss is:  tensor(7.7988, device='cuda:0')
current relative error is:  tensor(0.6519, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(7.6646, device='cuda:0')
current relative error is:  tensor(0.6360, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(7.5074, device='cuda:0')
current relative error is:  tensor(0.6519, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(7.7455, device='cuda:0')
current relative error is:  tensor(0.6513, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(7.5153, device='cuda:0')
current relative error is:  tensor(0.5857, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(7.4041, device='cuda:0')
current relative error is:  tensor(0.6119, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(7.337

...

current epoch is:  36600
current loss is:  tensor(1.0463, device='cuda:0')
current relative error is:  tensor(0.0773, device='cuda:0')
current epoch is:  36650
current loss is:  tensor(2.7541, device='cuda:0')
current relative error is:  tensor(0.1132, device='cuda:0')
current epoch is:  36700
current loss is:  tensor(1.0353, device='cuda:0')
current relative error is:  tensor(0.0820, device='cuda:0')
current epoch is:  36750
current loss is:  tensor(1.0029, device='cuda:0')
current relative error is:  tensor(0.0768, device='cuda:0')
current epoch is:  36800
current loss is:  tensor(1.0510, device='cuda:0')
current relative error is:  tensor(0.0761, device='cuda:0')
current epoch is:  36850
current loss is:  tensor(1.3519, device='cuda:0')
current relative error is:  tensor(0.0821, device='cuda:0')
current epoch is:  36900
current loss is:  tensor(1.0342, device='cuda:0')
current relative error is:  tensor(0.0785, device='cuda:0')
current epoch is:  36950
current loss is:  tensor(1.065

current epoch is:  39650
current loss is:  tensor(1.0303, device='cuda:0')
current relative error is:  tensor(0.0778, device='cuda:0')
current epoch is:  39700
current loss is:  tensor(1.1887, device='cuda:0')
current relative error is:  tensor(0.0779, device='cuda:0')
current epoch is:  39750
current loss is:  tensor(1.8991, device='cuda:0')
current relative error is:  tensor(0.0800, device='cuda:0')
current epoch is:  39800
current loss is:  tensor(1.0740, device='cuda:0')
current relative error is:  tensor(0.0738, device='cuda:0')
current epoch is:  39850
current loss is:  tensor(1.0087, device='cuda:0')
current relative error is:  tensor(0.0728, device='cuda:0')
current epoch is:  39900
current loss is:  tensor(1.1854, device='cuda:0')
current relative error is:  tensor(0.0771, device='cuda:0')
current epoch is:  39950
current loss is:  tensor(1.9864, device='cuda:0')
current relative error is:  tensor(0.1013, device='cuda:0')
current epoch is:  40000
current loss is:  tensor(0.993

current epoch is:  42700
current loss is:  tensor(1.0546, device='cuda:0')
current relative error is:  tensor(0.0745, device='cuda:0')
current epoch is:  42750
current loss is:  tensor(1.1300, device='cuda:0')
current relative error is:  tensor(0.0789, device='cuda:0')
current epoch is:  42800
current loss is:  tensor(1.3599, device='cuda:0')
current relative error is:  tensor(0.0914, device='cuda:0')
current epoch is:  42850
current loss is:  tensor(0.9450, device='cuda:0')
current relative error is:  tensor(0.0741, device='cuda:0')
current epoch is:  42900
current loss is:  tensor(0.9639, device='cuda:0')
current relative error is:  tensor(0.0760, device='cuda:0')
current epoch is:  42950
current loss is:  tensor(1.9882, device='cuda:0')
current relative error is:  tensor(0.0853, device='cuda:0')
current epoch is:  43000
current loss is:  tensor(0.9142, device='cuda:0')
current relative error is:  tensor(0.0704, device='cuda:0')
current epoch is:  43050
current loss is:  tensor(1.129

current epoch is:  45750
current loss is:  tensor(3.2700, device='cuda:0')
current relative error is:  tensor(0.1032, device='cuda:0')
current epoch is:  45800
current loss is:  tensor(1.0097, device='cuda:0')
current relative error is:  tensor(0.0732, device='cuda:0')
current epoch is:  45850
current loss is:  tensor(2.2596, device='cuda:0')
current relative error is:  tensor(0.0783, device='cuda:0')
current epoch is:  45900
current loss is:  tensor(1.5996, device='cuda:0')
current relative error is:  tensor(0.0866, device='cuda:0')
current epoch is:  45950
current loss is:  tensor(0.9064, device='cuda:0')
current relative error is:  tensor(0.0686, device='cuda:0')
current epoch is:  46000
current loss is:  tensor(0.8712, device='cuda:0')
current relative error is:  tensor(0.0690, device='cuda:0')
current epoch is:  46050
current loss is:  tensor(1.2430, device='cuda:0')
current relative error is:  tensor(0.0693, device='cuda:0')
current epoch is:  46100
current loss is:  tensor(2.007

current epoch is:  48800
current loss is:  tensor(1.1958, device='cuda:0')
current relative error is:  tensor(0.0729, device='cuda:0')
current epoch is:  48850
current loss is:  tensor(0.8537, device='cuda:0')
current relative error is:  tensor(0.0692, device='cuda:0')
current epoch is:  48900
current loss is:  tensor(1.5007, device='cuda:0')
current relative error is:  tensor(0.0831, device='cuda:0')
current epoch is:  48950
current loss is:  tensor(0.8734, device='cuda:0')
current relative error is:  tensor(0.0691, device='cuda:0')
current epoch is:  49000
current loss is:  tensor(1.1684, device='cuda:0')
current relative error is:  tensor(0.0709, device='cuda:0')
current epoch is:  49050
current loss is:  tensor(0.8877, device='cuda:0')
current relative error is:  tensor(0.0724, device='cuda:0')
current epoch is:  49100
current loss is:  tensor(0.8576, device='cuda:0')
current relative error is:  tensor(0.0674, device='cuda:0')
current epoch is:  49150
current loss is:  tensor(1.301